In [22]:
import os
import duckdb
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(usecwd=False))
BASE_DIR = Path(os.environ["BASE_DIR"])
print(f'Base dir: {BASE_DIR}')

DB_PATH = f"{BASE_DIR}/data/reddit.duckdb"

con = duckdb.connect(str(DB_PATH))

Base dir: /home/martina/Desktop/Git/reddit-llm


In [3]:
import pandas as pd

df_nasdaq = pd.read_csv('https://datahub.io/core/nasdaq-listings/r/nasdaq-listed-symbols.csv')
df_nyse = pd.read_csv('https://datahub.io/core/nyse-other-listings/r/nyse-listed.csv')

display(df_nasdaq)

,Symbol,Company Name,Security Name,Market Category,Test Issue,Financial Status,Round Lot Size,ETF,NextShares
0,AACB,Artius II Acquisition Inc.,Artius II Acquisition Inc. - Class A Ordinary ...,G,N,N,100.0,N,N
1,AACBR,Artius II Acquisition Inc.,Artius II Acquisition Inc. - Rights,G,N,N,100.0,N,N
2,AACBU,Artius II Acquisition Inc.,Artius II Acquisition Inc. - Units,G,N,N,100.0,N,N
3,AACG,ATA Creativity Global,ATA Creativity Global - American Depositary Sh...,S,N,N,100.0,N,N
4,AADR,AdvisorShares Dorsey Wright ADR ETF,AdvisorShares Dorsey Wright ADR ETF,G,N,N,100.0,Y,N
...,...,...,...,...,...,...,...,...,...
5247,ZXZZT,NASDAQ TEST STOCK,NASDAQ TEST STOCK,G,Y,N,100.0,N,N
5248,ZYBT,Zhengye Biotechnology Holding Limited,Zhengye Biotechnology Holding Limited - Ordina...,S,N,N,100.0,N,N
5249,ZYME,Zymeworks Inc.,Zymeworks Inc. - Common Stock,Q,N,N,100.0,N,N
5250,File Creation Time: 1231202518:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Remove the last 2 weird rows
df_nasdaq = df_nasdaq[:-2]

In [5]:
display(df_nyse)

,ACT Symbol,Company Name
0,A,"Agilent Technologies, Inc. Common Stock"
1,AA,Alcoa Corporation Common Stock
2,AAM,AA Mission Acquisition Corp. Class A Ordinary ...
3,AAM.U,"AA Mission Acquisition Corp. Units, each consi..."
4,AAM.W,"AA Mission Acquisition Corp. Warrants, each wh..."
...,...,...
2879,ZTO,ZTO Express (Cayman) Inc. American Depositary ...
2880,ZTR,Virtus Total Return Fund Inc.
2881,ZTS,Zoetis Inc. Class A Common Stock
2882,ZVIA,Zevia PBC Class A Common Stock


In [9]:
# Rename ticker to match the other table
df_nyse.rename(columns={'ACT Symbol': 'Symbol'}, inplace=True)
df_nyse

,Symbol,Company Name
0,A,"Agilent Technologies, Inc. Common Stock"
1,AA,Alcoa Corporation Common Stock
2,AAM,AA Mission Acquisition Corp. Class A Ordinary ...
3,AAM.U,"AA Mission Acquisition Corp. Units, each consi..."
4,AAM.W,"AA Mission Acquisition Corp. Warrants, each wh..."
...,...,...
2879,ZTO,ZTO Express (Cayman) Inc. American Depositary ...
2880,ZTR,Virtus Total Return Fund Inc.
2881,ZTS,Zoetis Inc. Class A Common Stock
2882,ZVIA,Zevia PBC Class A Common Stock


In [10]:
df_all_tickers = pd.concat([df_nasdaq, df_nyse], ignore_index=True)
df_all_tickers

,Symbol,Company Name,Security Name,Market Category,Test Issue,Financial Status,Round Lot Size,ETF,NextShares
0,AACB,Artius II Acquisition Inc.,Artius II Acquisition Inc. - Class A Ordinary ...,G,N,N,100.0,N,N
1,AACBR,Artius II Acquisition Inc.,Artius II Acquisition Inc. - Rights,G,N,N,100.0,N,N
2,AACBU,Artius II Acquisition Inc.,Artius II Acquisition Inc. - Units,G,N,N,100.0,N,N
3,AACG,ATA Creativity Global,ATA Creativity Global - American Depositary Sh...,S,N,N,100.0,N,N
4,AADR,AdvisorShares Dorsey Wright ADR ETF,AdvisorShares Dorsey Wright ADR ETF,G,N,N,100.0,Y,N
...,...,...,...,...,...,...,...,...,...
8129,ZTO,ZTO Express (Cayman) Inc. American Depositary ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8130,ZTR,Virtus Total Return Fund Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8131,ZTS,Zoetis Inc. Class A Common Stock,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8132,ZVIA,Zevia PBC Class A Common Stock,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
con.sql('''CREATE TABLE tickers AS 
              SELECT DISTINCT Symbol as ticker
              FROM df_all_tickers''')

In [14]:
con.table("tickers").show()

┌────────────┐
│   ticker   │
│  varchar   │
├────────────┤
│ AALG       │
│ AAOI       │
│ AAON       │
│ AAPU       │
│ ABNG       │
│ ABTS       │
│ ABUS       │
│ ACWX       │
│ ADAMN      │
│ ADBE       │
│  ·         │
│  ·         │
│  ·         │
│ UTF        │
│ VIPS       │
│ VLTO       │
│ VNO$M      │
│ WD         │
│ WFC        │
│ WH         │
│ WLKP       │
│ XYF        │
│ ZETA       │
├────────────┤
│ 8133 rows  │
│ (20 shown) │
└────────────┘



In [16]:
con.sql('SHOW ALL TABLES;')

┌──────────┬─────────┬─────────────────┬─────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────┬───────────┐
│ database │ schema  │      name       │                            column_names                             │                           column_types                            │ temporary │
│ varchar  │ varchar │     varchar     │                              varchar[]                              │                             varchar[]                             │  boolean  │
├──────────┼─────────┼─────────────────┼─────────────────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────┼───────────┤
│ reddit   │ main    │ comment_tickers │ [comment_id, ticker, direction, confidence]                         │ [VARCHAR, VARCHAR, VARCHAR, DOUBLE]                               │ false     │
│ reddit   │ main    │ comments        │ [com

In [23]:
con.sql('''SELECT *
        FROM comment_tickers ct
        LIMIT 100;
        ''').show()

┌────────────┬─────────┬───────────┬────────────┐
│ comment_id │ ticker  │ direction │ confidence │
│  varchar   │ varchar │  varchar  │   double   │
├────────────┼─────────┼───────────┼────────────┤
│ ny4pzbq    │ NICE    │ neutral   │        0.0 │
│ ny4pzbq    │ ON      │ neutral   │        0.0 │
│ ny4b09v    │ WAY     │ neutral   │        0.0 │
│ ny4b09v    │ MOVE    │ neutral   │        0.0 │
│ ny4b09v    │ UP      │ neutral   │        0.0 │
│ ny4b09v    │ ON      │ neutral   │        0.0 │
│ ny4b09v    │ ANY     │ neutral   │        0.0 │
│ ny4jcy6    │ A       │ neutral   │        0.0 │
│ ny4jcy6    │ IT      │ neutral   │        0.0 │
│ ny4jcy6    │ UP      │ neutral   │        0.0 │
│    ·       │ ·       │    ·      │         ·  │
│    ·       │ ·       │    ·      │         ·  │
│    ·       │ ·       │    ·      │         ·  │
│ ny4i8ge    │ IT      │ neutral   │        0.0 │
│ ny4i8ge    │ UP      │ neutral   │        0.0 │
│ ny4i8ge    │ ALL     │ neutral   │        0.0 │


In [17]:
# Top tickers by mentions
con.sql('''SELECT
            ct.ticker,
            COUNT(*) AS mentions
        FROM comment_tickers ct
        JOIN comments c ON ct.comment_id = c.comment_id
        WHERE c.created_utc >= NOW() - INTERVAL '1 day'
        GROUP BY ct.ticker
        ORDER BY mentions DESC
        LIMIT 10;
        ''').show()

In [ ]:
# Bullis vs bearish
con.sql('''SELECT
            ticker,
            SUM(direction = 'bullish') AS bullish,
            SUM(direction = 'bearish') AS bearish
        FROM comment_tickers ct
        JOIN comments c ON ct.comment_id = c.comment_id
        WHERE c.created_utc >= NOW() - INTERVAL '1 day'
        GROUP BY ticker
        ORDER BY bullish DESC;
        ''').show()

In [ ]:
# Weighted by comments
con.sql('''SELECT
    ct.ticker,
    SUM(c.score) AS weighted_mentions
FROM comment_tickers ct
JOIN comments c ON ct.comment_id = c.comment_id
WHERE c.created_utc >= NOW() - INTERVAL '1 day'
GROUP BY ct.ticker
ORDER BY weighted_mentions DESC
LIMIT 10;
        ''').show()

In [18]:
con.close()